In [1]:
###############################################################################
# The Institute for the Design of Advanced Energy Systems Integrated Platform
# Framework (IDAES IP) was produced under the DOE Institute for the
# Design of Advanced Energy Systems (IDAES).
#
# Copyright (c) 2018-2025 by the software owners: The Regents of the
# University of California, through Lawrence Berkeley National Laboratory,
# National Technology & Engineering Solutions of Sandia, LLC, Carnegie Mellon
# University, West Virginia University Research Corporation, et al.
# All rights reserved.  Please see the files COPYRIGHT.md and LICENSE.md
# for full copyright and license information.
#
###############################################################################

# Degeneracy Hunter Examples
Author: Prof. Alex Dowling (adowling@nd.edu), University of Notre Dame  
Maintainer: Andrew Lee  
Updated: 2023-06-01  

> Alexander W. Dowling and Lorenz T. Biegler (2015). Degeneracy Hunter: An Algorithm for Determining Irreducible Sets of Degenerate Constraints in Mathematical Programs. 12th International Symposium on Process Systems Engineering and 25th European Symposium on Computer Aided Process Engineering. Ed. by Krist V. Gernaey, Jakob K. Huusom, and Raqul Gani. Computer-Aided Chemical Engineering, Vol. 37, p. 809 – 814. [link to PDF](https://dowlinglab.nd.edu/assets/447116/2015_degeneracy_hunter_an_algorithm_for_determining_irreducible_sets_of_degenerate_constraints_in_mathematical_prog.pdf)

This notebook shows how to use the Degeneracy Hunter tool which is part of the IDAES Diagnostics Toolbox features using two motivating examples:

* Inspect constraint violations and bounds of a Pyomo model
* Compute the Irreducible Degenerate Set (IDS) for a Pyomo model
* Demonstrates the performance benefits in Ipopt from removing a single redundant constraint

<div class="alert alert-block alert-warning">
<b>Note:</b>
Degeneracy Hunter requires a suitable MILP solver. This example uses the open-source solver SCIP (https://scipopt.org/), however users may specify another solver of their choice if they have one available.
</div>

## What does Degeneracy Mean?

In simple terms, a degenerate model contains one or more constraints that do not add any additional information that is not already included in other constraints in the model. The simplest example of this is the case where a constraint in the model is duplicated. For example, consider the model below:

$$\begin{align*}
& x_1 + x_2 = 1 \\
& x_1 + x_2 = 1 \\
\end{align*} $$

Notice the two equality constraints are identical and thus redundant. This means the constraint qualifications that solvers rely on (e.g., [Linear Independence Constraint Qualification or LICQ](https://en.wikipedia.org/wiki/Karush%E2%80%93Kuhn%E2%80%93Tucker_conditions#Regularity_conditions_(or_constraint_qualifications))) do not hold which has three important implications:

1. The optimal solution may not be mathematically well-defined (e.g., the dual variables are not unique)
2. The calculations performed by the optimization solver may become numerically poorly scaled
3. Theoretical convergence properties of optimization algorithms may not hold

Put another way, for a deterministic (square) problem we required that the number of equality constraints be equal to the number of free variables. In this case, there appear to be two equality constraints but in reality there is effectively only one as the two constraints are the same. Thus, the problem is not well defined and there exist a family of solutions that can satisfy the constraints given; any combination of values for ``x1`` and ``x2`` that sum to one will satisfy the constraints as written.

In practice, degeneracies are rarely as straight-forward as a duplicated constraint. A more common occurrence is a set of linearly dependent constraints; that is a system of constraints where one constraint in the set can be formed by a linear combination of other constraints in the set. For example:

$$\begin{align*}
& x_1 + x_2 + x_3 = 1 \\
& 2x_1 + 2x_2 + 2x_3 = 2 \\
\end{align*} $$

Here, the second constraint can be formed by multiplying the first constraint by two. Another common example of linearly dependent constraints is the combination of a set of material balances for all components in a system and an overall material balance. In this case, the overall material balances can be formed by adding all the individual component balances together. Depending on the number of components being modeled, this system of constraints can be quite large, but the end result is the same; the model effectively has one less constraint than it would appear to have.

The absolute best defense against this is to detect degenerate equations and reformulate the model to remove them; this is the purpose of Degeneracy Hunter. Let's see it in action.

##  Setup

We start by importing Pyomo and the IDAES Diagnostics Toolbox.

In [2]:
import pyomo.environ as pyo

from idaes.core.util.model_diagnostics import DiagnosticsToolbox

In [3]:
from idaes.core.util.testing import _enable_scip_solver_for_testing

scip_solver = pyo.SolverFactory("scip")
if not scip_solver.available():
    _enable_scip_solver_for_testing()
assert scip_solver.available()

## Example: Linear Program with Redundant Equality Constraints

Let's apply these tools to a poorly formulated optimization problem:

$$\begin{align*} \min_{\mathbf{x}} \quad & \sum_{i=\{1,...,3\}} x_i \\
\mathrm{s.t.}~~& x_1 + x_2 \geq 1 \\
& x_1 + x_2 + x_3 = 1 \\
& x_2 - 2 x_3 \leq 1 \\
& x_1 + x_3 \geq 1 \\
& x_1 + x_2 + x_3 = 1 \\
\end{align*} $$

As you can see, this is similar to our example above with duplicated equality constraints.

The cell below shows how to construct this model in Pyomo.

In [4]:
def build_example(include_redundant_constraint=True):
    m = pyo.ConcreteModel()

    m.I = pyo.Set(initialize=[i for i in range(1, 4)])

    m.x = pyo.Var(m.I, bounds=(0, 5), initialize=1.0)

    m.con1 = pyo.Constraint(expr=m.x[1] + m.x[2] >= 1)
    m.con2 = pyo.Constraint(expr=m.x[1] + m.x[2] + m.x[3] == 1)
    m.con3 = pyo.Constraint(expr=m.x[2] - 2 * m.x[3] <= 1)
    m.con4 = pyo.Constraint(expr=m.x[1] + m.x[3] >= 1)

    if include_redundant_constraint:
        m.con5 = pyo.Constraint(expr=m.x[1] + m.x[2] + m.x[3] == 1)

    m.obj = pyo.Objective(expr=sum(m.x[i] for i in m.I))

    return m


m = build_example()

### Check for Structural Issues

Before we try to solve the model, the first thing we should do is use the Diagnostics Toolbox to check for any structural issues in our model. The cell below creates an instance of the Diagnostics Toolbox and calls the ``report_structural_issues`` method.

In [5]:
dt = DiagnosticsToolbox(m)
dt.report_structural_issues()

As you can see, the toolbox is reporting one degree of freedom and a structural singularity. In this case, these are both expected as we are trying to solve an optimization problem with one degree of freedom so we can move on to the next step. 

### Evaluate the initial point

Before we try to solve our degenerate model, it can often be useful to check the state of the model at the initial step seen by the solver. The cell below creates an instance of Ipopt and sets the iteration limit to 0 so that it terminates at the initial condition.

In [6]:
solver = pyo.SolverFactory("ipopt")

# Specifying an iteration limit of 0 allows us to inspect the initial point
solver.options["max_iter"] = 0

# "Solving" the model with an iteration limit of 0 load the initial point and applies
# any preprocessors (e.g., enforces bounds)
solver.solve(m, tee=True)

### Identify constraints with large residuals at the initial point

With the solution at the initial point, we can then use the Diagnostics Toolbox to check the residuals of all constraints at the initial point.

In [7]:
# Check for large residuals
dt.display_constraints_with_large_residuals()

We can also check for variables which are close to (or outside of) their bounds.

In [8]:
dt.display_variables_near_bounds()

### Solve the optimization problem and extract the solution

There appear to be no significant issues at the initial point, so let us move on to solving the optimization problem.

In [9]:
solver.options["max_iter"] = 50
solver.solve(m, tee=True)

We got lucky here. Ipopt implements several algorithmic and numerical safeguards to handle (mildy) degenerate equations. Nevertheless, notice the last column of the Ipopt output labeled `ls`. This is the number of line search evaluations. For iterations 0 to 11, `ls` is 1, which means Ipopt is taking full steps. For iterations 12 to 16, however, `ls` is greater than 20. This means Ipopt is struggling (a little) to converge to the solution.

The cell below shows the values of the variables at the solution.

In [10]:
m.x.display()

### Check the rank of the Jacobian of the equality constraints

One way to check for the presence of degenerate equations is to calculate the rank of the Jacobian matrix of the equality constraints. A singular value near 0 indicates the Jacobian is rank deficient, and for each near-zero singular value there is likely one degenerate constraint.

The Diagnostics Toolbox contains a Singular Value Decomposition (SVD) toolbox which can be used to determine the number of small singular values in the Jacobian for a model as shown below.

In [11]:
svd = dt.prepare_svd_toolbox()
svd.display_rank_of_equality_constraints()

As can be seen above, there is one singular value less than 1e-6, suggesting we have one degenerate constraint.

### Finding Irreducible Degenerate Sets (IDS)

Next, we can use Degeneracy Hunter to identify irreducible degenerate sets; that is, the smallest set of constraints that contain a redundant constraint.

Degeneracy Hunter first identifies candidate degenerate equations by solving a mixed integer linear program (MILP). It then iterates through the candidate equations and solves a second MILP for each to compute the irreducible degenerate set that must contain the candidate equation.

<div class="alert alert-block alert-warning">
<b>Note:</b>
Degeneracy Hunter requires a suitable MILP solver, users can specify their solver of choice via a configuration argument. The default option is SCIP (https://scipopt.org/) which is an open-source solver.
</div>

The cell below shows how to create an instance of Degeneracy Hunter and generate a report of the irreducible degenerate sets.

In [12]:
dh = dt.prepare_degeneracy_hunter()
dh.report_irreducible_degenerate_sets()

As can be seen above, Degeneracy Hunter identified two constraints as potentially redundant (the duplicate constraints ``con2`` and ``con5``). Degeneracy Hunter then reports an Irreducible Degenerate Set for each of these constraints, which in the case as identical. More complex models may have partially overlapping IDS's.

These results show us that ``con2`` and ``con5`` are mutually redundant; i.e., we can eliminate one of these with no effect on the model solution.

### Reformulate Model to Remove Redundant Constraint

Now let's reformulate the model by removing the redundant equality constraint ``con5`` and solve the reformulated model.

In [13]:
m2 = build_example(include_redundant_constraint=False)

solver.solve(m2, tee=True)

First, let us check to see if the optimal solution has changed.

In [14]:
m2.x.display()

We get the same answer as before, but careful inspection of the Ipopt output reveals a subtle improvement. Notice `ls` is only 1 or 2 for all of the iterations, in contrast to more than 20 for the original model. This means Ipopt is taking (nearly) full steps for all iterations.

Let's also compare the number of function evaluations.

Original model (using Ipopt 3.13.2 with `ma27`):
```
Number of objective function evaluations             = 111
Number of objective gradient evaluations             = 17
Number of equality constraint evaluations            = 111
Number of inequality constraint evaluations          = 111
Number of equality constraint Jacobian evaluations   = 17
Number of inequality constraint Jacobian evaluations = 17
Number of Lagrangian Hessian evaluations             = 16
```

Reformulated model (using Ipopt 3.13.2 with `ma27`):
```
Number of objective function evaluations             = 23
Number of objective gradient evaluations             = 18
Number of equality constraint evaluations            = 23
Number of inequality constraint evaluations          = 23
Number of equality constraint Jacobian evaluations   = 18
Number of inequality constraint Jacobian evaluations = 18
Number of Lagrangian Hessian evaluations             = 17
```

Removing a **single redundant constraint** reduced the number of objective and constraint evaluations by a **factor of 5**!

Often degenerate equations have a much worse impact on large-scale problems; for example, degenerate equations can cause Ipopt to require many more iterations or terminate at an infeasible point.


In [15]:
import pytest

assert pyo.value(m2.x[1]) == pytest.approx(1, rel=1e-6)
assert pyo.value(m2.x[2]) == pytest.approx(0, abs=1e-6)
assert pyo.value(m2.x[3]) == pytest.approx(0, abs=1e-6)